# Import modules and define functions for baselines

In [ ]:
import gurobipy as gp
env = gp.Env(empty=True)
env.setParam('OutputFlag', 0)
env.start()
# Create a new model

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = '16'

In [ ]:
import subprocess
from dsd import fibheap, dsp
import dsd
from scipy import optimize
import networkx as nx
import numpy as np
import time
import warnings
import math
warnings.filterwarnings('ignore')

## LP-S

The output x is further used for LP-G

In [ ]:
def LP_solve(model, G, c, theta, verbose=False, obj_lower_bound=0):
    start_time = time.time()
    n = G.number_of_nodes()
    m = G.number_of_edges()
    global_opt = 0
    global_S = []
    
    xs, ys = [], []
    for i in range(n):
        tmp_v = model.addVar(lb=0, ub=1, vtype=gp.GRB.CONTINUOUS, name="x_"+str(i))
        xs.append(tmp_v)
    for e in G.edges():
        tmp_e = model.addVar(lb=0, ub=1, vtype=gp.GRB.CONTINUOUS, name="y_"+str(e[0])+"_"+str(e[1]))
        ys.append(tmp_e)
    # Set the objective function
    model.setObjective(sum(ys), gp.GRB.MAXIMIZE)

    # Add constraints
    cnt = 0
    for e in G.edges():
        model.addConstr(ys[cnt]-xs[e[0]] <= 0)
        model.addConstr(ys[cnt]-xs[e[1]] <= 0)
        cnt += 1
    model.addConstr(sum(xs)<=1)
    cu_sum = sum([xs[u]*c[u] for u in G.nodes()])
    model.addConstr(-cu_sum <= -theta)

    # Optimize the model
    model.optimize()
    
    if verbose:
        if model.status == gp.GRB.OPTIMAL:
            print("Optimal solution found!")
            for v in model.getVars():
                print(f"{v.varName}: {v.x}")
            print(f"Objective value: {model.objVal}")
        else:
            print("No solution found.")

    optimal = 0
    opt_S = []
    x = [i.x for i in model.getVars()[:n]]
    # y = res['x'][n:]
    rs = set(x)
    x = np.array(x)
    for r in rs:
        S = np.where(x>=r)[0]
        
        cur_density = float(G.subgraph(S).number_of_edges())/len(S)
        cur_nw = sum(c[u] for u in S)/len(S)
        # print(cur_density, cur_nw)
        if cur_nw>theta and cur_density>optimal:
            optimal = cur_density
            opt_S = list(S)
            
    runningtime = time.time()-start_time
    return optimal, opt_S, runningtime, x

## Integer programming that solve the problem exactly (extremely slow)

In [ ]:
def IP_solve(G, c, theta, verbose=False, obj_lower_bound=0):
    start_time = time.time()
    n = G.number_of_nodes()
    m = G.number_of_edges()
    global_opt = 0
    global_S = []
    for size in range(2,n+1):
        model = gp.Model(env=env)
        xs, ys = [], []
        for i in range(n):
            tmp_v = model.addVar(lb=0, ub=1, vtype=gp.GRB.INTEGER, name="x_"+str(i))
            xs.append(tmp_v)
        for e in G.edges():
            tmp_e = model.addVar(lb=0, ub=1, vtype=gp.GRB.INTEGER, name="y_"+str(e[0])+"_"+str(e[1]))
            ys.append(tmp_e)
        # Set the objective function
        model.setObjective(sum(ys)/size, gp.GRB.MAXIMIZE)

        # Add constraints
        cnt = 0
        for e in G.edges():
            model.addConstr(ys[cnt]-xs[e[0]] <= 0)
            model.addConstr(ys[cnt]-xs[e[1]] <= 0)
            cnt += 1
        model.addConstr(sum(xs)==size)
        cu_sum = sum([xs[u]*c[u] for u in G.nodes()])
        model.addConstr(-cu_sum/size <= -theta)

        # Optimize the model
        model.optimize()

        if verbose:
            if model.status == gp.GRB.OPTIMAL:
                print("Optimal solution found!")
                for v in model.getVars():
                    print(f"{v.varName}: {v.x}")
                print(f"Objective value: {model.objVal}")
            else:
                print("No solution found.")

        optimal = 0
        opt_S = []
        
        if abs(model.objVal)>optimal:
            optimal = abs(model.objVal)
            opt_S = list(np.where(np.array([i.x for i in model.getVars()[:n]])>0.5)[0])
            
    runningtime = time.time()-start_time
    return optimal, opt_S, runningtime

## AF baseline

In [ ]:
def baseline(G, c, theta):
    S = np.where(c>theta)[0]
    densest_S, opt_d = dsd.exact_densest(G.subgraph(S))
    return densest_S, opt_d

# Exp

In [ ]:
lp_times, lp_d = [],[]
lp_extend = []
flow_times, flow_d = [],[]
peel_times, peel_d = [],[]
base_times, base_d = [],[]
multi=6
for n in [200,400,800, 1600, 3200]:
    p=multi*math.log(n)/n
    tlp_times, tlp_d = [],[]
    tlp_extend = []
    tflow_times, tflow_d = [],[]
    tpeel_times, tpeel_d = [],[]
    tbase_times, tbase_d = [],[]
    print("!!!!",p,"!!!!")
    for iteration in range(10):
        G = nx.stochastic_block_model([n,n], [[p, 3/n],[3/n,1.2*p]])
        n_w_1 = np.clip(np.random.normal(0.5,0.05,n), -1, 1) 
        n_w_2 = np.clip(np.random.normal(0.3,0.05,n), -1, 1) 
        nw = np.concatenate([n_w_1, n_w_2])

        model = gp.Model(env=env)
        lp_opt, lp_S, lp_time,x = LP_solve(model, G, nw, 0.45, verbose=False)
        tlp_times.append(lp_time)
        tlp_d.append(lp_opt)
        print("LP-S", lp_opt)

        S = np.where(x>0)[0]
        while  np.average(nw[S])<0.45:
            remove = min(S, key=lambda e:nw[e])
            S = S[S!=remove]
        tlp_extend.append(G.subgraph(S).number_of_edges()/G.subgraph(S).number_of_nodes())
        print("LP-G", tlp_extend[-1])

        f = open('data/syn/sbm_200.txt','w')
        f.write(str(G.number_of_nodes())+' '+str(G.number_of_edges())+'\n')

        for i in n_w_1:
            f.write(str(int(i*100))+'\n')
        for i in n_w_2:
            f.write(str(int(i*100))+'\n')

        for e in G.edges():
            f.write(str(e[0])+' '+str(e[1])+' '+'1\n')

        f.close()

        st = time.time()
        fout = open(f"syn_sbm_process_{n}.flow", "w")
        subprocess.run(["./src/exactweighted","100", "100", "45", "100", "data/syn/sbm_200.txt", "syn_sbm.maxflow"], stdout=fout) 
        finish_time = time.time()-st
        f = open("syn_sbm.maxflow","r")
        uids = []
        for line in f:
            uids.append(int(line.strip()))
        d = G.subgraph(uids).number_of_edges()/len(uids)
        tflow_times.append(finish_time)
        tflow_d.append(d)
        print(d, len(uids))

        st = time.time()
        subprocess.run(["./src/ip_a", "45", "1000", "1000", "data/syn/sbm_200.txt", "syn_sbm.peel"],
            stdout = subprocess.DEVNULL) 
        finish_time = time.time()-st
        f = open("syn_sbm.peel","r")
        uids = []
        for line in f:
            uids.append(int(line.strip()))
        d = G.subgraph(uids).number_of_edges()/len(uids)
        tpeel_times.append(finish_time)
        tpeel_d.append(d)
        print(d)
        b_S, b_d = baseline(G, nw, 0.45)
        tbase_d.append(b_d)
        print("baseline", b_d)
    lp_times.append((np.average(tlp_times),np.std(tlp_times)))
    lp_d.append((np.average(tlp_d),np.std(tlp_d)))
    
    lp_extend.append((np.average(tlp_extend),np.std(tlp_extend)))
    
    flow_times.append((np.average(tflow_times),np.std(tflow_times)))
    flow_d.append((np.average(tflow_d),np.std(tflow_d)))
    
    base_d.append((np.average(tbase_d),np.std(tbase_d)))
    
    peel_times.append((np.average(tpeel_times),np.std(tpeel_times)))
    peel_d.append((np.average(tpeel_d),np.std(tpeel_d)))

## plot

In [ ]:
lvs = 5
plt.errorbar(range(lvs),[i[0] for i in lp_d],[i[1] for i in lp_d],label='LP-S',marker='2',color='grey', capsize=10)
plt.errorbar(range(lvs),[i[0] for i in lp_extend],[i[1] for i in lp_extend],label='LP-G',marker='2',color='y', capsize=10)
plt.errorbar(range(lvs),[i[0] for i in base_d],[i[1] for i in base_d],label='AF',marker='o',color='g', capsize=10)
plt.errorbar(range(lvs),[i[0] for i in flow_d],[i[1] for i in flow_d],label='Q-DISCO-Lagr',marker='x',color='r', capsize=10)
plt.errorbar(range(lvs),[i[0] for i in peel_d],[i[1] for i in peel_d],label='Q-DISCO-Peel',marker='d',color='b', capsize=10)
plt.xticks(range(lvs),[200,400,800,1600,3200])
plt.xlabel('block size')
plt.ylabel('density')
plt.legend( fontsize="15", ncol=1)
plt.show()

In [ ]:
# for iteration in range(5):
# lp_times, lp_d = [],[]
# lp_extend = []
# flow_times, flow_d = [],[]
# peel_times, peel_d = [],[]
# base_times, base_d = [],[]
for n in [3200]:
    print("!!!!",n,"!!!!")
    tlp_times, tlp_d = [],[]
    tlp_extend = []
    tflow_times, tflow_d = [],[]
    tpeel_times, tpeel_d = [],[]
    tbase_times, tbase_d = [],[]
    for iteration in range(5):
        G = nx.stochastic_block_model([n,n], [[2*math.log(n)/n, 3/n],[3/n,math.log(n)/100]])
        n_w_1 = np.clip(np.random.normal(0.5,0.01,n), -1, 1) 
        n_w_2 = np.clip(np.random.normal(0.3,0.1,n), -1, 1) 
        nw = np.concatenate([n_w_1, n_w_2])
        # plt.hist([n_w_1, n_w_2])
        # plt.show()

        model = gp.Model(env=env)
        lp_opt, lp_S, lp_time,x = LP_solve(model, G, nw, 0.4, verbose=False)
        tlp_times.append(lp_time)
        tlp_d.append(lp_opt)
        print("LP", lp_opt)

        S = np.where(x>0)[0]
        while  np.average(nw[S])<0.4:
            remove = min(S, key=lambda e:nw[e])
            S = S[S!=remove]
        tlp_extend.append(G.subgraph(S).number_of_edges()/G.subgraph(S).number_of_nodes())

        f = open('data/syn/sbm_200.txt','w')
        f.write(str(G.number_of_nodes())+' '+str(G.number_of_edges())+'\n')

        for i in n_w_1:
            f.write(str(int(i*100))+'\n')
        for i in n_w_2:
            f.write(str(int(i*100))+'\n')

        for e in G.edges():
            f.write(str(e[0])+' '+str(e[1])+' '+'1\n')

        f.close()

        st = time.time()
        fout = open(f"output/syn/sbm_200_process_{n}.flow", "w")
        subprocess.run(["./code-greedy++/exactweighted","100", "100", "40", "100", "data/syn/sbm_200.txt", "output/syn/sbm_200.maxflow"], stdout=fout) 
        finish_time = time.time()-st
        f = open("output/syn/sbm_200.maxflow","r")
        uids = []
        for line in f:
            uids.append(int(line.strip()))
        if (len(uids)<=0):
            print("!!!")
            break
        d = G.subgraph(uids).number_of_edges()/len(uids)
        tflow_times.append(finish_time)
        tflow_d.append(d)
        print(d, len(uids))

        st = time.time()
        subprocess.run(["./code-greedy++/ip", "40", "100", "100", "data/syn/sbm_200.txt", "output/syn/sbm_200.peel"],
            stdout = subprocess.DEVNULL) 
        finish_time = time.time()-st
        f = open("output/syn/sbm_200.peel","r")
        uids = []
        for line in f:
            uids.append(int(line.strip()))
        d = G.subgraph(uids).number_of_edges()/len(uids)
        tpeel_times.append(finish_time)
        tpeel_d.append(d)
        print(d)
        b_S, b_d = baseline(G, nw, 0.4)
        tbase_d.append(b_d)
    lp_times.append((np.average(tlp_times),np.std(tlp_times)))
    lp_d.append((np.average(tlp_d),np.std(tlp_d)))
    
    lp_extend.append((np.average(tlp_extend),np.std(tlp_extend)))
    
    flow_times.append((np.average(tflow_times),np.std(tflow_times)))
    flow_d.append((np.average(tflow_d),np.std(tflow_d)))
    
    # base_times.append((np.average(tbase_times),np.std(tbase_times)))
    base_d.append((np.average(tbase_d),np.std(tbase_d)))
    
    peel_times.append((np.average(tpeel_times),np.std(tpeel_times)))
    peel_d.append((np.average(tpeel_d),np.std(tpeel_d)))

### test two plans of algorithm 2

In [ ]:
a_times, a_d = [],[]
b_times, b_d = [],[]
for n in [100,200,400,800,1600,3200]:
    print("!!!!",n,"!!!!")
    ta_times, ta_d = [],[]
    tb_times, tb_d = [],[]
    for iteration in range(5):
        G = nx.stochastic_block_model([n,n], [[2*math.log(n)/n, 3/n],[3/n,math.log(n)/100]])
        n_w_1 = np.clip(np.random.normal(0.5,0.01,n), -1, 1) 
        n_w_2 = np.clip(np.random.normal(0.3,0.1,n), -1, 1) 
        nw = np.concatenate([n_w_1, n_w_2])
        
        f = open('data/syn/sbm_200.txt','w')
        f.write(str(G.number_of_nodes())+' '+str(G.number_of_edges())+'\n')

        for i in n_w_1:
            f.write(str(int(i*100))+'\n')
        for i in n_w_2:
            f.write(str(int(i*100))+'\n')

        for e in G.edges():
            f.write(str(e[0])+' '+str(e[1])+' '+'1\n')

        f.close()

        fouta = open(f"output/syn/sbm_200_process_{n}.pa", "w")
        
        st = time.time()
        subprocess.run(["./code-greedy++/ip_a", "40", "100", "100", "data/syn/sbm_200.txt", "output/syn/sbm_200.pa"],
            stdout = fouta) 
        finish_time = time.time()-st
        f = open("output/syn/sbm_200.pa","r")
        uids = []
        for line in f:
            uids.append(int(line.strip()))
        d = G.subgraph(uids).number_of_edges()/len(uids)
        ta_times.append(finish_time)
        ta_d.append(d)
        print(d)
        
        foutb = open(f"output/syn/sbm_200_process_{n}.pb", "w")
        
        st = time.time()
        subprocess.run(["./code-greedy++/ip_b", "40", "100", "100", "data/syn/sbm_200.txt", "output/syn/sbm_200.pb"],
            stdout = foutb) 
        finish_time = time.time()-st
        f = open("output/syn/sbm_200.pb","r")
        uids = []
        for line in f:
            uids.append(int(line.strip()))
        d = G.subgraph(uids).number_of_edges()/len(uids)
        tb_times.append(finish_time)
        tb_d.append(d)
        print(d)
    
    a_times.append((np.average(ta_times),np.std(ta_times)))
    a_d.append((np.average(ta_d),np.std(ta_d)))
    b_times.append((np.average(tb_times),np.std(tb_times)))
    b_d.append((np.average(tb_d),np.std(tb_d)))

In [ ]:
tlp_extend, tlp_d

In [ ]:
lvs = 6
plt.errorbar(range(lvs),[i[0] for i in lp_d],[i[1] for i in lp_d],label='LP-S',marker='2',color='b', capsize=10)
plt.errorbar(range(lvs),[i[0] for i in lp_extend],[i[1] for i in lp_extend],label='LP-G',marker='2',color='y', capsize=10)
plt.errorbar(range(lvs),[i[0] for i in base_d],[i[1] for i in base_d],label='AF',marker='o',color='g', capsize=10)
plt.errorbar(range(lvs),[i[0] for i in flow_d],[i[1] for i in flow_d],label='Algo-1',marker='x',color='r', capsize=10)
plt.errorbar(range(lvs),[i[0] for i in peel_d],[i[1] for i in peel_d],label='Algo-2',marker='d',color='grey', capsize=10)
# plt.errorbar(range(lvs),[i[0] for i in a_d],[i[1] for i in a_d],label='Algo-a',marker='x',color='r', capsize=10)
# plt.errorbar(range(lvs),[i[0] for i in b_d],[i[1] for i in b_d],label='Algo-b',marker='d',color='c', capsize=10)
plt.xticks(range(lvs),[100,200,400,800,1600,3200])
plt.xlabel('block size')
plt.ylabel('density')
plt.legend()
plt.tight_layout()
plt.savefig("fig/syn_density.png")
plt.show()

In [ ]:
lvs = 6
plt.plot(range(lvs),[i[0] for i in lp_times],label='LP',color='b',marker='2')
plt.plot(range(lvs),np.array([i[0] for i in flow_times])/12,label='AF',color='g',marker='o')
plt.plot(range(lvs),[i[0] for i in flow_times],label='Algo-1',color='r',marker='x')
plt.plot(range(lvs),[i[0] for i in peel_times],label='Algo-2',color='grey',marker='d')
# plt.errorbar(range(lvs),[i[1] for i in lp_extend],[i[1] for i in lp_extend],label='LP-G')
plt.xticks(range(lvs),[100,200,400,800,1600,3200])
plt.ylabel("time (s)")
plt.xlabel('block size')
plt.yscale('log')
# plt.legend()
plt.tight_layout()
plt.savefig("fig/syn_time.png")
plt.show()

In [ ]:
import pickle 

In [ ]:
(lp_times,flow_times,peel_times) = pickle.load(open('output/syn/time.pickle','rb'))
(lp_d,flow_d,peel_d,base_d,lp_extend) = pickle.load(open('output/syn/density.pickle','rb'))

In [ ]:


pickle.dump((lp_times,flow_times,peel_times), open('output/syn/time.pickle','wb'))

In [ ]:
pickle.dump((lp_d,flow_d,peel_d,base_d,lp_extend), open('output/syn/density.pickle','wb'))

In [ ]:
peel_times

In [ ]:
lvs = 3
plt.errorbar(range(lvs),[i[0] for i in lp_d],[i[1] for i in lp_d],label='LP',marker='2')
plt.errorbar(range(lvs),[i[0] for i in flow_d],[i[1] for i in flow_d],label='flow',marker='x')
plt.errorbar(range(lvs),[i[0] for i in peel_d],[i[1] for i in peel_d],label='peel',marker='d')
plt.errorbar(range(lvs),[i[0] for i in base_d],[i[1] for i in base_d],label='baseline',marker='1')
plt.errorbar(range(lvs),[i[0] for i in lp_extend],[i[1] for i in lp_extend],label='LP-extend')
plt.legend()

In [ ]:
d, lp_opt

In [ ]:
flow_d

In [ ]:
S = np.where(np.array([i.x for i in model.getVars()[:2*n]])>0)[0]
cur_density = float(G.subgraph(S).number_of_edges())/len(S)
cur_nw = sum(nw[u] for u in S)/len(S)
cur_density, cur_nw

In [ ]:
cur_density, cur_nw

In [ ]:
plt.plot(lp_d,label='LP',marker='2')
plt.plot(flow_d,label='flow',marker='x')
plt.plot(peel_d,label='peel',marker='d')
plt.plot(base_d,label='baseline',marker='1')
plt.plot(lp_extend,label='LP-extend')
plt.xticks(range(5),[100,200,400,800,1600])
plt.xlabel('block size')
plt.ylabel('density')
plt.legend()
plt.tight_layout()
plt.savefig("fig/syn_density.png")
plt.show()

In [ ]:
plt.plot(lp_times,label='LP',marker='2')
plt.plot(flow_times,label='flow',marker='x')
plt.plot(peel_times,label='peel',marker='d')
plt.plot(np.array(flow_times)/12,label='baseline',marker='1')
plt.xticks(range(6),[100,200,400,800,1600,3200])
plt.xlabel('block size')
plt.ylabel("time (s)")
plt.yscale('log')
plt.legend()
plt.tight_layout()
plt.savefig("fig/syn_time.png")
plt.show()

In [ ]:
fout = open("output/syn/sbm_200_process.flow","r")
lmd = []
e_d, n_d = [],[]
for line in fout:
    if line.startswith("using lambda"):
        lmd.append(int(line.split(" ")[-1].strip()))
    if line.startswith("size:"):
        e_d.append(float(line.split(":")[-1].strip()))
        n_d.append(float(line.split(",")[-2].split(":")[-1].strip()))

In [ ]:
e_d, n_d, lmd

In [ ]:
for idx in range(3,len(e_d)):
    L = 125
    edge_d = e_d[idx]
    node_d = n_d[idx]
    l_upper = lmd[idx]
    plt.plot([0,L],[edge_d, edge_d+ L*(node_d/100-0.4)])
#     , label="lambda="+lbd+", size="+size
plt.ylim(-0.1,20)
plt.xlabel(r"$\lambda$")
# plt.yscale("log")
plt.tight_layout()
plt.savefig("fig/flow_lagrangian_demo.png")
plt.show()

In [ ]:
f = open('data/syn/sbm_200.txt','w')
f.write(str(G.number_of_nodes())+' '+str(G.number_of_edges())+'\n')

for i in n_w_1:
    f.write(str(int(i*100))+'\n')
for i in n_w_2:
    f.write(str(int(i*100))+'\n')
    
for e in G.edges():
    f.write(str(e[0])+' '+str(e[1])+' '+'1\n')

f.close()

In [ ]:
st = time.time()
subprocess.run(["./code-greedy++/exactweighted","1000", "10", "40", "100", "data/syn/sbm_200.txt", "output/syn/sbm_200.maxflow"],
    stdout = subprocess.DEVNULL) 
finish_time = time.time()-st

f = open("output/syn/sbm_200.maxflow","r")
uids = []
for line in f:
    uids.append(int(line.strip()))
d = G.subgraph(uids).number_of_edges()/len(uids)


st = time.time()
subprocess.run(["./code-greedy++/ip", "40", "100", "100", "data/syn/sbm_200.txt", "output/syn/sbm_200.peel"],
    stdout = subprocess.DEVNULL) 
finish_time = time.time()-st

f = open("output/syn/sbm_200.peel","r")
uids = []
for line in f:
    uids.append(int(line.strip()))
d = G.subgraph(uids).number_of_edges()/len(uids)

In [ ]:

%%bash
./code-greedy++/exactweighted 1000 10 40 100 data/syn/sbm_200.txt output/syn/sbm_200.maxflow

In [ ]:
%%bash
./code-greedy++/ip 40 100 100 data/syn/sbm_200.txt output/syn/sbm_200.peel